In [1]:
import numpy as np
import torch
import tensorflow as tf
import time

In [2]:
w = np.random.rand(1_000_000)
x = np.random.rand(1_000_000)

## 1. With python for loop

In [3]:
%%time
tic = time.time()

for j in range(100):
    z=0
    for i in range(1_000_000):
        z += w[i] * x[i]

toc = time.time()

time_for_loop = (toc - tic) * 10
print(f"For loop: {time_for_loop} ms.")

For loop: 193.084979057312 ms.
CPU times: user 19.3 s, sys: 7.04 ms, total: 19.3 s
Wall time: 19.3 s


## 2. With numpy vectorization

In [15]:
%%time
tic = time.time()

for j in range(100):
    z=0
    z = np.dot(w, x)

toc = time.time()

time_for_numpy = (toc - tic) * 10
print(f"Numpy vec: {time_for_numpy} ms.")

Numpy vec: 0.11962652206420898 ms.
CPU times: user 215 ms, sys: 0 ns, total: 215 ms
Wall time: 12.1 ms


## 3. With Tensorflow Tensor

In [9]:
w_tf = tf.convert_to_tensor(w)
x_tf = tf.convert_to_tensor(x)

In [10]:
%%time
tic = time.time()

for j in range(100):
    z=0
    z = tf.tensordot(w_tf, x_tf, axes=1)

toc = time.time()

time_for_tf = (toc - tic) * 10
print(f"tf vec: {time_for_tf} ms.")

tf vec: 0.44008970260620117 ms.
CPU times: user 41.6 ms, sys: 3.68 ms, total: 45.3 ms
Wall time: 44.1 ms


#### With opti

In [11]:
@tf.function
def dot(x, y):
    return tf.tensordot(x, y, axes=1)

In [24]:
%%time
tic = time.time()

for j in range(100):
    z=0
    z = dot(w_tf, x_tf)

toc = time.time()

time_for_tf_opti = (toc - tic) * 10
print(f"tf vec opti: {time_for_tf_opti} ms.")

tf vec opti: 0.17971515655517578 ms.
CPU times: user 20.8 ms, sys: 184 µs, total: 21 ms
Wall time: 18.1 ms


## 4. With Pytorch

In [25]:
w_torch = torch.tensor(w)
x_torch = torch.tensor(x)

In [30]:
%%time
tic = time.time()

for j in range(100):
    z=0
    z = torch.dot(w_torch, x_torch)

toc = time.time()

time_for_torch = (toc - tic) * 10
print(f"torch vec: {time_for_torch} ms.")

torch vec: 0.11466026306152344 ms.
CPU times: user 91.8 ms, sys: 1.12 ms, total: 92.9 ms
Wall time: 11.8 ms


In [31]:
times = [
    time_for_loop,
    time_for_numpy,
    time_for_tf,
    time_for_tf_opti,
    time_for_torch
]

<img src="img/tensor_vec.jpg" alt="drawing"/>